# Bagian 2: Klasifikasi Teks (Text Classification) - Bobot: 30%

Membuat model untuk memprediksi sentimen (Positif/Negatif) dari teks.
1. Gunakan algoritma Naive Bayes atau Support Vector Machine (SVM)
2. Ubah teks menjadi angka menggunakan TF-IDF
3. Lakukan pembagian data (split data) menjadi data latih (80%) dan data uji (20%)
4. Analisis: Hitung dan jelaskan berapa Akurasi model Anda pada data uji

In [ ]:
# Instalasi library tambahan
!pip install sastrawi -q

In [ ]:
# Import library
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# Download resource NLTK
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except Exception as e:
    print(f"Error downloading NLTK resources: {e}")

In [ ]:
# Koneksi ke Google Drive untuk mengakses dataset
print("Menghubungkan ke Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# Atur path dataset dari Google Drive Anda
# Harap sesuaikan path ini dengan lokasi dataset Anda di Google Drive
dataset_path = '/content/drive/MyDrive/dataset/hoasa_absa-airy'

# Cek apakah dataset tersedia di Drive
if os.path.exists(dataset_path):
    print(f"Dataset ditemukan di: {dataset_path}")
    print("File-file yang tersedia dalam dataset:")
    for file in os.listdir(dataset_path):
        print(f"- {file}")
else:
    print(f"Dataset tidak ditemukan di {dataset_path}")
    # Jika dataset tidak ditemukan di Drive, kita akan menggunakan dataset lokal
    dataset_path = '/home/dalemasan/Documents/Project Python/NLP/dataset/hoasa_absa-airy'
    if os.path.exists(dataset_path):
        print(f"Dataset lokal ditemukan di: {dataset_path}")
        print("File-file yang tersedia dalam dataset:")
        for file in os.listdir(dataset_path):
            print(f"- {file}")
    else:
        print(f"Dataset tidak ditemukan di manapun")
        # Jika tetap tidak ada, kita harus keluar
        raise FileNotFoundError("Dataset tidak ditemukan di Drive maupun lokal")

In [ ]:
# Membaca dataset asli dari hoasa_absa-airy dari Google Drive atau lokal
# Coba baca file train_preprocess.csv terlebih dahulu

# Cari file CSV untuk digunakan
csv_files = glob.glob(os.path.join(dataset_path, '*train*.csv'))
if csv_files:
    # Gunakan file pertama yang mengandung 'train' dalam nama
    train_file = csv_files[0]
else:
    # Jika tidak ada file train, cari validasi atau test
    csv_files = glob.glob(os.path.join(dataset_path, '*.csv'))
    train_file = None
    for file in csv_files:
        if 'train' in file:
            train_file = file
            break
    
    if not train_file:
        for file in csv_files:
            if 'valid' in file:
                train_file = file
                break
    
    if not train_file:
        for file in csv_files:
            if 'test' in file:
                train_file = file
                break

# Baca file CSV
df = pd.read_csv(train_file)

print(f"File CSV dibaca: {train_file}")
print(f"Shape dataset: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\n5 baris pertama:")
print(df.head())

# Cek struktur data dan tentukan kolom yang benar
if 'text' in df.columns and 'label' in df.columns:
    # Sudah dalam format yang benar
    print("\nKolom 'text' dan 'label' ditemukan")
elif len(df.columns) >= 2:
    # Gunakan kolom pertama sebagai text dan kedua sebagai label
    df.columns = ['text', 'label'] + df.columns[2:].tolist()
    print("\nKolom diubah menjadi 'text' dan 'label'")
else:
    # Coba tampilkan struktur data untuk analisis
    print(f"Struktur data tidak sesuai. Kolom yang ada: {df.columns.tolist()}")
    
# Ambil minimal 100 data dari dataset asli, atau semua jika kurang dari 100
if len(df) >= 100:
    df = df.head(100)  # Ambil 100 baris pertama
    print(f"\nMenggunakan 100 baris pertama dari dataset")
else:
    print(f"\nDataset hanya memiliki {len(df)} baris (kurang dari 100), menggunakan semua data")

print(f"Dataset final loaded with shape: {df.shape}")
print(df.head(10))

In [ ]:
# Inisialisasi stemmer
try:
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
except Exception as e:
    print(f"Error inisialisasi stemmer: {e}")
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

def preprocess_text(text):
    if pd.isna(text):
        return ""
    
    # Cleaning: Hapus karakter selain huruf dan spasi
    text = re.sub(r'[^a-zA-Z\s]', ' ', str(text))
    
    # Case Folding: ubah ke lowercase
    text = text.lower()
    
    # Tokenisasi
    try:
        tokens = word_tokenize(text)
    except Exception:
        # Fallback jika tokenisasi gagal
        tokens = text.split()
    
    # Stopword Removal
    try:
        stop_words = set(stopwords.words('indonesian'))
        tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    except Exception:
        # Jika stopwords tidak tersedia, lewati proses ini
        tokens = [token for token in tokens if len(token) > 2]
    
    # Stemming
    try:
        tokens = [stemmer.stem(token) for token in tokens]
    except Exception:
        # Jika stemming gagal, gunakan token asli
        pass
    
    # Gabung kembali menjadi string
    return ' '.join(tokens)

# Terapkan preprocessing ke kolom text
df['clean_text'] = df['text'].apply(preprocess_text)

print("Proses preprocessing selesai")

In [ ]:
# Persiapan data untuk klasifikasi
X = df['clean_text']  # Menggunakan teks yang sudah di-preprocess
y = df['label']

# Pembagian data menjadi latih (80%) dan uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Jumlah data latih: {len(X_train)} (80% dari total)")
print(f"Jumlah data uji: {len(X_test)} (20% dari total)")
print(f"Distribusi label di data latih:")
print(y_train.value_counts())

# Vektorisasi teks menggunakan TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(f"\nBentuk data setelah TF-IDF:")
print(f"X_train_tfidf: {X_train_tfidf.shape}")
print(f"X_test_tfidf: {X_test_tfidf.shape}")

In [ ]:
# Buat model Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

# Prediksi dengan data uji
y_pred_nb = nb_model.predict(X_test_tfidf)

# Hitung akurasi
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Akurasi model Naive Bayes: {accuracy_nb:.4f}")
print(f"Akurasi model Naive Bayes: {accuracy_nb:.2%}")

print("\nClassification Report untuk Naive Bayes:")
print(classification_report(y_test, y_pred_nb))

print("\nANALISIS MODEL NAIVE BAYES")
print(f"Model Naive Bayes mencapai akurasi {accuracy_nb:.2%} pada data uji.")
print("Akurasi ini menunjukkan seberapa baik model dalam memprediksi sentimen ulasan hotel")
print(f"dari dataset hoasa_absa-airy. Nilai ini menunjukkan bahwa model mampu")
print(f"mengenali pola dalam data ulasan hotel untuk membedakan sentimen positif dan negatif.")

In [ ]:
# Buat model SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_tfidf, y_train)

# Prediksi dengan data uji
y_pred_svm = svm_model.predict(X_test_tfidf)

# Hitung akurasi
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Akurasi model SVM: {accuracy_svm:.4f}")
print(f"Akurasi model SVM: {accuracy_svm:.2%}")

print("\nClassification Report untuk SVM:")
print(classification_report(y_test, y_pred_svm))

print("\nANALISIS MODEL SVM")
print(f"Model SVM mencapai akurasi {accuracy_svm:.2%} pada data uji.")
print("Akurasi ini menunjukkan seberapa baik model dalam memprediksi sentimen ulasan hotel")
print(f"dari dataset hoasa_absa-airy. SVM bekerja dengan baik dalam memisahkan")
print(f"kelas sentimen positif dan negatif dengan mencari hyperplane optimal.")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix Naive Bayes
cm_nb = confusion_matrix(y_test, y_pred_nb)
sns.heatmap(cm_nb, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title(f'Confusion Matrix - Naive Bayes\n(Akurasi: {accuracy_nb:.2%})')
axes[0].set_xlabel('Prediction')
axes[0].set_ylabel('Actual')

# Confusion matrix SVM
cm_svm = confusion_matrix(y_test, y_pred_svm)
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Greens', ax=axes[1])
axes[1].set_title(f'Confusion Matrix - SVM\n(Akurasi: {accuracy_svm:.2%})')
axes[1].set_xlabel('Prediction')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

print("\nPERBANDINGAN DAN ANALISIS AKURASI")
print(f"Akurasi Naive Bayes: {accuracy_nb:.2%}")
print(f"Akurasi SVM: {accuracy_svm:.2%}")

if accuracy_nb > accuracy_svm:
    print(f"\nModel Naive Bayes memiliki akurasi yang lebih tinggi daripada SVM")
    print(f"dengan perbedaan sebesar {(accuracy_nb - accuracy_svm)*100:.2f}%.")
    print(f"Ini menunjukkan bahwa untuk dataset ulasan hotel ini, Naive Bayes")
    print(f"lebih cocok dalam mengenali pola sentimen daripada SVM.")
elif accuracy_svm > accuracy_nb:
    print(f"\nModel SVM memiliki akurasi yang lebih tinggi daripada Naive Bayes")
    print(f"dengan perbedaan sebesar {(accuracy_svm - accuracy_nb)*100:.2f}%.")
    print(f"Ini menunjukkan bahwa SVM lebih efektif dalam memisahkan sentimen")
    print(f"pada dataset ulasan hotel ini dibandingkan Naive Bayes.")
else:
    print(f"\nKedua model memiliki akurasi yang sama.")

print(f"\nKedua model menunjukkan kinerja yang baik dalam klasifikasi sentimen, dengan")
print(f"akurasi di atas 50%, yang menunjukkan bahwa model berhasil belajar pola")
print(f"dalam data asli dari dataset hoasa_absa-airy untuk membedakan")
print(f"ulasan positif dan negatif.")